#Load Libraries

In [ ]:
# Load Libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd 
import multiprocessing
import random
import seaborn as sns


import csv
import requests
from bs4 import BeautifulSoup
from pandas_profiling import ProfileReport
import pandas_profiling
from sklearn.preprocessing import scale
from sklearn.preprocessing import OneHotEncoder
import timeit

#Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X= pd.read_csv('/content/drive/My Drive/TFM_DF/dt_c3_2_cat1_sin_outliers.csv', index_col=0)
X

,purchase_id,"q_Aceite, vinagre y sal",q_Aceitunas y encurtidos,q_Acondicionador y mascarilla,q_Afeitado y cuidado para hombre,q_Agua,q_Alimentación infantil,q_Arreglos,q_Arroz,q_Arroz y pasta,q_Atún y otras conservas de pescado,q_Aves y jamón cocido,q_Aves y pollo,q_Azúcar y edulcorante,q_Bacón y salchichas,q_Bases de maquillaje y corrector,q_Berberechos y mejillones,"q_Biberón, chupete y menaje",q_Bollería de horno,q_Bollería envasada,q_Bífidus,q_Cacao soluble y chocolate a la taza,q_Café cápsula y monodosis,q_Café molido y en grano,q_Café soluble y otras bebidas,q_Carne,q_Carne congelada,q_Cerdo,q_Cereales,q_Cerveza,q_Cerveza sin alcohol,q_Champú,q_Chicles y caramelos,q_Chocolate,q_Chopped y mortadela,q_Coloración cabello,q_Colorete y polvos,q_Conejo y cordero,q_Conservas de verdura y frutas,q_Cuidado corporal,...,percp_Sidra y cava,percp_Sopa y caldo,percp_Sushi,percp_Tartas y churros,percp_Tartas y pasteles,percp_Tinto de verano y sangría,percp_Toallitas y pañales,percp_Tomate,percp_Tomate y otros sabores,percp_Tortitas,percp_Turrones,percp_Té e infusiones,percp_Tónica y bitter,percp_Utensilios de limpieza y calzado,percp_Vacuno,percp_Velas y decoración,percp_Verdura,percp_Vino blanco,percp_Vino lambrusco y espumoso,percp_Vino rosado,percp_Vino tinto,percp_Yogures desnatados,percp_Yogures griegos,percp_Yogures líquidos,percp_Yogures naturales y sabores,percp_Yogures y postres infantiles,count_creation_card,count_card_update,count_card_country,Spain,datetime_diff_sec,datetime_diff_min,regular_customer,year,month,above_average,quarter,season,datetime_dif_card_sec,datetime_dif_card_min
564,53326,2,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,256.92,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,50.99,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,1,1,1,1,1201.0,20.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
1056,54662,3,0,0,0,1,0,1,0,0,0,1,2,1,1,0,0,0,0,0,1,0,0,1,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,377.91,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,28.49,0.00,0.0,0.0,0.0,45.93,0.00,0.0,0.00,0.0,1,1,1,1,170997.0,2849.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
1058,54664,3,0,0,0,1,0,1,0,0,0,1,2,1,1,0,0,0,0,0,1,0,0,1,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,382.35,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,28.82,0.00,0.0,0.0,0.0,46.47,0.00,0.0,0.00,0.0,1,1,1,1,171199.0,2853.0,1,2018,8,1,q3,Summer,39817351.0,663622.0
1581,55924,6,0,6,6,14,0,0,12,0,0,6,14,2,0,0,0,0,0,0,4,6,6,0,8,0,2,0,0,0,0,0,6,18,0,2,0,0,0,2,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,76.67,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,2,2,2,0,-99950.0,-1666.0,1,2018,8,1,q3,Summer,39362777.0,656046.0
1586,55938,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,27.41,0.00,0.0,0.0,0.0,0.00,65.48,0.0,0.00,0.0,1,1,1,1,354946.0,5915.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524375,5316196,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,36.86,0.0,0.0,0.00,0.0,0.0,0.0,0.0,16.03,96.15,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,1,1,1,1,1163254.0,19387.0,1,2020,5,0,q2,Spring,0.0,0.0
524376,5316215,0,12,0,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,27.11,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,117.47,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,2,2,1,1,61621138.0,1027018.0,0,2020,5,1,q2,Spring,45258918.0,754315.0
524377,5316223,0,0,0,0,5,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00

In [ ]:
X.head(10)

,purchase_id,"q_Aceite, vinagre y sal",q_Aceitunas y encurtidos,q_Acondicionador y mascarilla,q_Afeitado y cuidado para hombre,q_Agua,q_Alimentación infantil,q_Arreglos,q_Arroz,q_Arroz y pasta,q_Atún y otras conservas de pescado,q_Aves y jamón cocido,q_Aves y pollo,q_Azúcar y edulcorante,q_Bacón y salchichas,q_Bases de maquillaje y corrector,q_Berberechos y mejillones,"q_Biberón, chupete y menaje",q_Bollería de horno,q_Bollería envasada,q_Bífidus,q_Cacao soluble y chocolate a la taza,q_Café cápsula y monodosis,q_Café molido y en grano,q_Café soluble y otras bebidas,q_Carne,q_Carne congelada,q_Cerdo,q_Cereales,q_Cerveza,q_Cerveza sin alcohol,q_Champú,q_Chicles y caramelos,q_Chocolate,q_Chopped y mortadela,q_Coloración cabello,q_Colorete y polvos,q_Conejo y cordero,q_Conservas de verdura y frutas,q_Cuidado corporal,...,percp_Sidra y cava,percp_Sopa y caldo,percp_Sushi,percp_Tartas y churros,percp_Tartas y pasteles,percp_Tinto de verano y sangría,percp_Toallitas y pañales,percp_Tomate,percp_Tomate y otros sabores,percp_Tortitas,percp_Turrones,percp_Té e infusiones,percp_Tónica y bitter,percp_Utensilios de limpieza y calzado,percp_Vacuno,percp_Velas y decoración,percp_Verdura,percp_Vino blanco,percp_Vino lambrusco y espumoso,percp_Vino rosado,percp_Vino tinto,percp_Yogures desnatados,percp_Yogures griegos,percp_Yogures líquidos,percp_Yogures naturales y sabores,percp_Yogures y postres infantiles,count_creation_card,count_card_update,count_card_country,Spain,datetime_diff_sec,datetime_diff_min,regular_customer,year,month,above_average,quarter,season,datetime_dif_card_sec,datetime_dif_card_min
564,53326,2,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,...,0.00,0.00,0.0,0.0,256.92,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,50.99,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,1,1,1,1,1201.0,20.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
1056,54662,3,0,0,0,1,0,1,0,0,0,1,2,1,1,0,0,0,0,0,1,0,0,1,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,1,...,0.00,0.00,0.0,0.0,377.91,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,28.49,0.0,0.0,0.0,0.0,45.93,0.00,0.00,0.0,0.00,1,1,1,1,170997.0,2849.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
1058,54664,3,0,0,0,1,0,1,0,0,0,1,2,1,1,0,0,0,0,0,1,0,0,1,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,1,...,0.00,0.00,0.0,0.0,382.35,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,28.82,0.0,0.0,0.0,0.0,46.47,0.00,0.00,0.0,0.00,1,1,1,1,171199.0,2853.0,1,2018,8,1,q3,Summer,39817351.0,663622.0
1581,55924,6,0,6,6,14,0,0,12,0,0,6,14,2,0,0,0,0,0,0,4,6,6,0,8,0,2,0,0,0,0,0,6,18,0,2,0,0,0,2,...,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,76.67,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,2,2,2,0,-99950.0,-1666.0,1,2018,8,1,q3,Summer,39362777.0,656046.0
1586,55938,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,27.41,0.0,0.0,0.0,0.0,0.00,65.48,0.00,0.0,0.00,1,1,1,1,354946.0,5915.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
1796,56464,0,0,0,0,132,0,132,220,0,0,0,198,0,0,0,0,0,0,0,0,0,0,0,0,0,0,176,0,44,0,0,0,0,0,0,0,44,0,0,...,0.00,2166.67,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,2875.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,3966.67,0.0,0.00,22,22,8,0,-108.0,-2.0,1,2018,8,1,q3,Summer,39393641.0,656560.0
1847,56599,0,0,1,0,1,0,0,1,0,1,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,1,0,0,2,0,...,0.00,0.00,0.0,0.0,386.90,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,183.33,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,1,1,1,1,434078.0,7234.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
1848,56600,0,0,1,0,1,0,0,1,0,1,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,1,0,0,2,0,...,0.00,0.00,0.0,0.0,445.21,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,210.96,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,1,1,1,1,434185.0,7236.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
1936,56820,2,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2649.25,0.00,0.0,0.0,0.00,0.0,0.0,50.75,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,81

In [ ]:
X=X.drop(columns= ['purchase_id', 'Spain'])
X

,"q_Aceite, vinagre y sal",q_Aceitunas y encurtidos,q_Acondicionador y mascarilla,q_Afeitado y cuidado para hombre,q_Agua,q_Alimentación infantil,q_Arreglos,q_Arroz,q_Arroz y pasta,q_Atún y otras conservas de pescado,q_Aves y jamón cocido,q_Aves y pollo,q_Azúcar y edulcorante,q_Bacón y salchichas,q_Bases de maquillaje y corrector,q_Berberechos y mejillones,"q_Biberón, chupete y menaje",q_Bollería de horno,q_Bollería envasada,q_Bífidus,q_Cacao soluble y chocolate a la taza,q_Café cápsula y monodosis,q_Café molido y en grano,q_Café soluble y otras bebidas,q_Carne,q_Carne congelada,q_Cerdo,q_Cereales,q_Cerveza,q_Cerveza sin alcohol,q_Champú,q_Chicles y caramelos,q_Chocolate,q_Chopped y mortadela,q_Coloración cabello,q_Colorete y polvos,q_Conejo y cordero,q_Conservas de verdura y frutas,q_Cuidado corporal,q_Cuidado e higiene facial,...,percp_Salazones y ahumados,percp_Sidra y cava,percp_Sopa y caldo,percp_Sushi,percp_Tartas y churros,percp_Tartas y pasteles,percp_Tinto de verano y sangría,percp_Toallitas y pañales,percp_Tomate,percp_Tomate y otros sabores,percp_Tortitas,percp_Turrones,percp_Té e infusiones,percp_Tónica y bitter,percp_Utensilios de limpieza y calzado,percp_Vacuno,percp_Velas y decoración,percp_Verdura,percp_Vino blanco,percp_Vino lambrusco y espumoso,percp_Vino rosado,percp_Vino tinto,percp_Yogures desnatados,percp_Yogures griegos,percp_Yogures líquidos,percp_Yogures naturales y sabores,percp_Yogures y postres infantiles,count_creation_card,count_card_update,count_card_country,datetime_diff_sec,datetime_diff_min,regular_customer,year,month,above_average,quarter,season,datetime_dif_card_sec,datetime_dif_card_min
564,2,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,256.92,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,50.99,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,1,1,1,1201.0,20.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
1056,3,0,0,0,1,0,1,0,0,0,1,2,1,1,0,0,0,0,0,1,0,0,1,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,377.91,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,28.49,0.00,0.0,0.0,0.0,45.93,0.00,0.0,0.00,0.0,1,1,1,170997.0,2849.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
1058,3,0,0,0,1,0,1,0,0,0,1,2,1,1,0,0,0,0,0,1,0,0,1,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,382.35,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,28.82,0.00,0.0,0.0,0.0,46.47,0.00,0.0,0.00,0.0,1,1,1,171199.0,2853.0,1,2018,8,1,q3,Summer,39817351.0,663622.0
1581,6,0,6,6,14,0,0,12,0,0,6,14,2,0,0,0,0,0,0,4,6,6,0,8,0,2,0,0,0,0,0,6,18,0,2,0,0,0,2,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,76.67,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,2,2,2,-99950.0,-1666.0,1,2018,8,1,q3,Summer,39362777.0,656046.0
1586,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,27.41,0.00,0.0,0.0,0.0,0.00,65.48,0.0,0.00,0.0,1,1,1,354946.0,5915.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524375,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,36.86,0.0,0.0,0.00,0.0,0.0,0.0,0.0,16.03,96.15,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,1,1,1,1163254.0,19387.0,1,2020,5,0,q2,Spring,0.0,0.0
524376,0,12,0,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,27.11,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,117.47,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,2,2,1,61621138.0,1027018.0,0,2020,5,1,q2,Spring,45258918.0,754315.0
524377,0,0,0,0,5,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0

In [ ]:
f= X[X['Class']==1]
f

,"q_Aceite, vinagre y sal",q_Aceitunas y encurtidos,q_Acondicionador y mascarilla,q_Afeitado y cuidado para hombre,q_Agua,q_Alimentación infantil,q_Arreglos,q_Arroz,q_Arroz y pasta,q_Atún y otras conservas de pescado,q_Aves y jamón cocido,q_Aves y pollo,q_Azúcar y edulcorante,q_Bacón y salchichas,q_Bases de maquillaje y corrector,q_Berberechos y mejillones,"q_Biberón, chupete y menaje",q_Bollería de horno,q_Bollería envasada,q_Bífidus,q_Cacao soluble y chocolate a la taza,q_Café cápsula y monodosis,q_Café molido y en grano,q_Café soluble y otras bebidas,q_Carne,q_Carne congelada,q_Cerdo,q_Cereales,q_Cerveza,q_Cerveza sin alcohol,q_Champú,q_Chicles y caramelos,q_Chocolate,q_Chopped y mortadela,q_Coloración cabello,q_Colorete y polvos,q_Conejo y cordero,q_Conservas de verdura y frutas,q_Cuidado corporal,q_Cuidado e higiene facial,...,percp_Salazones y ahumados,percp_Sidra y cava,percp_Sopa y caldo,percp_Sushi,percp_Tartas y churros,percp_Tartas y pasteles,percp_Tinto de verano y sangría,percp_Toallitas y pañales,percp_Tomate,percp_Tomate y otros sabores,percp_Tortitas,percp_Turrones,percp_Té e infusiones,percp_Tónica y bitter,percp_Utensilios de limpieza y calzado,percp_Vacuno,percp_Velas y decoración,percp_Verdura,percp_Vino blanco,percp_Vino lambrusco y espumoso,percp_Vino rosado,percp_Vino tinto,percp_Yogures desnatados,percp_Yogures griegos,percp_Yogures líquidos,percp_Yogures naturales y sabores,percp_Yogures y postres infantiles,count_creation_card,count_card_update,count_card_country,datetime_diff_sec,datetime_diff_min,regular_customer,year,month,above_average,quarter,season,datetime_dif_card_sec,datetime_dif_card_min
564,2,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.00,0.0,0.00,256.92,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,50.99,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,1,1,1,1201.0,20.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
1056,3,0,0,0,1,0,1,0,0,0,1,2,1,1,0,0,0,0,0,1,0,0,1,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,1,0,...,0.0,0.0,0.00,0.0,0.00,377.91,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,28.49,0.0,0.0,0.0,0.0,45.93,0.00,0.00,0.00,0.0,1,1,1,170997.0,2849.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
1058,3,0,0,0,1,0,1,0,0,0,1,2,1,1,0,0,0,0,0,1,0,0,1,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,1,0,...,0.0,0.0,0.00,0.0,0.00,382.35,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,28.82,0.0,0.0,0.0,0.0,46.47,0.00,0.00,0.00,0.0,1,1,1,171199.0,2853.0,1,2018,8,1,q3,Summer,39817351.0,663622.0
1581,6,0,6,6,14,0,0,12,0,0,6,14,2,0,0,0,0,0,0,4,6,6,0,8,0,2,0,0,0,0,0,6,18,0,2,0,0,0,2,0,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,76.67,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,2,2,2,-99950.0,-1666.0,1,2018,8,1,q3,Summer,39362777.0,656046.0
1586,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,27.41,0.0,0.0,0.0,0.0,0.00,65.48,0.00,0.00,0.0,1,1,1,354946.0,5915.0,1,2018,8,0,q3,Summer,39817351.0,663622.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522703,0,0,0,0,16,0,40,0,0,0,0,60,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,4,4,4,6530678.0,108844.0,0,2020,5,1,q2,Spring,0.0,0.0
522705,0,0,0,0,0,0,60,30,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,6,6,3,8310146.0,138502.0,0,2020,5,1,q2,Spring,871242.0,14520.0
522707,0,0,0,0,6,0,30,0,0,0,0,45,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.00,0.0,0.00,0.00,0

In [ ]:
fraction_of_rows_f = f.sample(frac=0.3)
fraction_of_rows_f

,"q_Aceite, vinagre y sal",q_Aceitunas y encurtidos,q_Acondicionador y mascarilla,q_Afeitado y cuidado para hombre,q_Agua,q_Alimentación infantil,q_Arreglos,q_Arroz,q_Arroz y pasta,q_Atún y otras conservas de pescado,q_Aves y jamón cocido,q_Aves y pollo,q_Azúcar y edulcorante,q_Bacón y salchichas,q_Bases de maquillaje y corrector,q_Berberechos y mejillones,"q_Biberón, chupete y menaje",q_Bollería de horno,q_Bollería envasada,q_Bífidus,q_Cacao soluble y chocolate a la taza,q_Café cápsula y monodosis,q_Café molido y en grano,q_Café soluble y otras bebidas,q_Carne,q_Carne congelada,q_Cerdo,q_Cereales,q_Cerveza,q_Cerveza sin alcohol,q_Champú,q_Chicles y caramelos,q_Chocolate,q_Chopped y mortadela,q_Coloración cabello,q_Colorete y polvos,q_Conejo y cordero,q_Conservas de verdura y frutas,q_Cuidado corporal,q_Cuidado e higiene facial,...,percp_Salazones y ahumados,percp_Sidra y cava,percp_Sopa y caldo,percp_Sushi,percp_Tartas y churros,percp_Tartas y pasteles,percp_Tinto de verano y sangría,percp_Toallitas y pañales,percp_Tomate,percp_Tomate y otros sabores,percp_Tortitas,percp_Turrones,percp_Té e infusiones,percp_Tónica y bitter,percp_Utensilios de limpieza y calzado,percp_Vacuno,percp_Velas y decoración,percp_Verdura,percp_Vino blanco,percp_Vino lambrusco y espumoso,percp_Vino rosado,percp_Vino tinto,percp_Yogures desnatados,percp_Yogures griegos,percp_Yogures líquidos,percp_Yogures naturales y sabores,percp_Yogures y postres infantiles,count_creation_card,count_card_update,count_card_country,datetime_diff_sec,datetime_diff_min,regular_customer,year,month,above_average,quarter,season,datetime_dif_card_sec,datetime_dif_card_min
190654,0,0,5,30,15,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,30,10,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,307.69,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,982.42,0.0,0.00,0.00,0.0,0.00,5,5,4,-554347.0,-9240.0,1,2019,8,1,q3,Summer,5452628.0,90877.0
417404,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,3,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,53.23,0.0,26.45,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,95.16,0.00,0.00,51.61,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,35.16,1,1,1,2705080.0,45084.0,1,2020,1,1,q1,Winter,1311203.0,21853.0
390923,2,1,0,0,2,0,0,17,0,6,5,3,0,0,1,0,0,0,6,0,0,0,0,0,0,5,0,10,1,0,0,0,0,0,0,2,0,0,0,0,...,0.00,0.0,60.62,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,194.69,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,1,1,1,-480686.0,-8012.0,1,2020,1,1,q1,Winter,0.0,0.0
128218,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,45.00,0.00,0.00,0.00,0.00,83.89,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,1,1,1,-131.0,-3.0,1,2019,6,0,q2,Summer,11596445.0,193274.0
450313,6,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.0,0.00,0.0,122.62,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,1,1,1,-213.0,-4.0,1,2020,3,1,q1,Spring,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305156,24,0,30,0,30,6,6,36,0,6,12,24,12,42,6,0,6,0,0,6,6,0,6,0,0,0,36,6,0,0,24,0,6,0,0,0,0,12,6,30,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,616.67,0.0,276.67,0.0,0.0,130.00,0.00,493.33,0.00,506.67,200.00,0.00,0.0,0.0,0.00,0.0,0.00,553.33,0.0,380.00,6,6,3,1512017.0,25200.0,1,2019,11,1,q4,Autumn,1512052.0,25200.0
150026,3,0,1,0,12,0,0,2,0,0,1,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,14,4,0,0,1,0,0,0,0,0,1,0,0,1,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,141.98,0.00,30.86,0.00,0.0,0.0,0.00,0.0,0.00,79.42,0.0,0.00,1,1,1,8326389.0,138773.0,0,2019,7,1,q3,Summer,17934651.0,298910.0
137843,4,4,0,0,24,10,0,2,0,2,4,8,4,0,0,0,8,

In [ ]:
nf= X[X['Class']==0]
nf

,"q_Aceite, vinagre y sal",q_Aceitunas y encurtidos,q_Acondicionador y mascarilla,q_Afeitado y cuidado para hombre,q_Agua,q_Alimentación infantil,q_Arreglos,q_Arroz,q_Arroz y pasta,q_Atún y otras conservas de pescado,q_Aves y jamón cocido,q_Aves y pollo,q_Azúcar y edulcorante,q_Bacón y salchichas,q_Bases de maquillaje y corrector,q_Berberechos y mejillones,"q_Biberón, chupete y menaje",q_Bollería de horno,q_Bollería envasada,q_Bífidus,q_Cacao soluble y chocolate a la taza,q_Café cápsula y monodosis,q_Café molido y en grano,q_Café soluble y otras bebidas,q_Carne,q_Carne congelada,q_Cerdo,q_Cereales,q_Cerveza,q_Cerveza sin alcohol,q_Champú,q_Chicles y caramelos,q_Chocolate,q_Chopped y mortadela,q_Coloración cabello,q_Colorete y polvos,q_Conejo y cordero,q_Conservas de verdura y frutas,q_Cuidado corporal,q_Cuidado e higiene facial,...,percp_Salazones y ahumados,percp_Sidra y cava,percp_Sopa y caldo,percp_Sushi,percp_Tartas y churros,percp_Tartas y pasteles,percp_Tinto de verano y sangría,percp_Toallitas y pañales,percp_Tomate,percp_Tomate y otros sabores,percp_Tortitas,percp_Turrones,percp_Té e infusiones,percp_Tónica y bitter,percp_Utensilios de limpieza y calzado,percp_Vacuno,percp_Velas y decoración,percp_Verdura,percp_Vino blanco,percp_Vino lambrusco y espumoso,percp_Vino rosado,percp_Vino tinto,percp_Yogures desnatados,percp_Yogures griegos,percp_Yogures líquidos,percp_Yogures naturales y sabores,percp_Yogures y postres infantiles,count_creation_card,count_card_update,count_card_country,datetime_diff_sec,datetime_diff_min,regular_customer,year,month,above_average,quarter,season,datetime_dif_card_sec,datetime_dif_card_min
0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,106.59,0.00,0.00,0.0,0.0,0.0,0.0,62.02,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,115.89,0.00,0.0,1,1,1,-167.0,-3.0,1,2018,8,1,q3,Summer,39926283.0,665438.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,1,1,1,5312975.0,88549.0,0,2018,8,0,q3,Summer,45239259.0,753987.0
2,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,...,0.0,0.0,0.0,0.0,0.0,176.15,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,113.55,0.0,0.0,188.35,0.0,0.0,0.00,39.02,0.0,1,1,1,-2095.0,-35.0,1,2018,8,0,q3,Summer,39924075.0,665401.0
3,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,5,5,1,582890.0,9714.0,1,2018,8,0,q3,Summer,40508964.0,675149.0
4,0,0,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,2,2,1,3074318.0,51238.0,1,2018,8,1,q3,Summer,43000277.0,716671.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524375,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,36.86,0.0,0.0,0.0,0.0,0.00,0.0,0.0,16.03,96.15,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,1,1,1,1163254.0,19387.0,1,2020,5,0,q2,Spring,0.0,0.0
524376,0,12,0,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,27.11,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,117.47,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,2,2,1,61621138.0,1027018.0,0,2020,5,1,q2,Spring,45258918.0,754315.0
524377,0,0,0,0,5,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0

In [ ]:
fraction_of_rows_nf = nf.sample(frac=0.3)
fraction_of_rows_nf

,"q_Aceite, vinagre y sal",q_Aceitunas y encurtidos,q_Acondicionador y mascarilla,q_Afeitado y cuidado para hombre,q_Agua,q_Alimentación infantil,q_Arreglos,q_Arroz,q_Arroz y pasta,q_Atún y otras conservas de pescado,q_Aves y jamón cocido,q_Aves y pollo,q_Azúcar y edulcorante,q_Bacón y salchichas,q_Bases de maquillaje y corrector,q_Berberechos y mejillones,"q_Biberón, chupete y menaje",q_Bollería de horno,q_Bollería envasada,q_Bífidus,q_Cacao soluble y chocolate a la taza,q_Café cápsula y monodosis,q_Café molido y en grano,q_Café soluble y otras bebidas,q_Carne,q_Carne congelada,q_Cerdo,q_Cereales,q_Cerveza,q_Cerveza sin alcohol,q_Champú,q_Chicles y caramelos,q_Chocolate,q_Chopped y mortadela,q_Coloración cabello,q_Colorete y polvos,q_Conejo y cordero,q_Conservas de verdura y frutas,q_Cuidado corporal,q_Cuidado e higiene facial,...,percp_Salazones y ahumados,percp_Sidra y cava,percp_Sopa y caldo,percp_Sushi,percp_Tartas y churros,percp_Tartas y pasteles,percp_Tinto de verano y sangría,percp_Toallitas y pañales,percp_Tomate,percp_Tomate y otros sabores,percp_Tortitas,percp_Turrones,percp_Té e infusiones,percp_Tónica y bitter,percp_Utensilios de limpieza y calzado,percp_Vacuno,percp_Velas y decoración,percp_Verdura,percp_Vino blanco,percp_Vino lambrusco y espumoso,percp_Vino rosado,percp_Vino tinto,percp_Yogures desnatados,percp_Yogures griegos,percp_Yogures líquidos,percp_Yogures naturales y sabores,percp_Yogures y postres infantiles,count_creation_card,count_card_update,count_card_country,datetime_diff_sec,datetime_diff_min,regular_customer,year,month,above_average,quarter,season,datetime_dif_card_sec,datetime_dif_card_min
285709,4,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,11.76,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,2,2,1,-145.0,-3.0,1,2019,11,1,q4,Autumn,0.0,0.0
258115,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,1,1,1,8926767.0,148779.0,0,2019,10,0,q4,Autumn,10366748.0,172779.0
401951,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,30.50,0.0,0.0,0.0,278.51,0.0,0.00,0.0,0.0,0.0,1,1,1,11425582.0,190426.0,0,2020,1,1,q1,Winter,4844918.0,80748.0
224085,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,101.38,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,2,2,1,17693117.0,294885.0,0,2019,9,0,q3,Autumn,21217243.0,353620.0
256379,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,1,1,1,8525641.0,142094.0,0,2019,10,1,q4,Autumn,10087483.0,168124.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142345,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,1,1,1,11915424.0,198590.0,0,2019,7,0,q3,Summer,22210894.0,370181.0
477142,1,0,0,0,0,0,0,0,0,0,4,3,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7,0,0,0,2,0,1,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83.85,0.0,0.00,0.0,0.0,0.0,0.0,43.48,0.0,0.0,55.28,0.0,0.0,0.0,142.86,0.0,80.12,0.0,67.7,0.0,1,1,1,-79.0,-2.0,1,2020,3,1,q1,Spring,0.0,0.0
405409,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,16.34,0.0,0.0,0.0,

In [ ]:
X2= fraction_of_rows_f.append(fraction_of_rows_nf)
X2

,"q_Aceite, vinagre y sal",q_Aceitunas y encurtidos,q_Acondicionador y mascarilla,q_Afeitado y cuidado para hombre,q_Agua,q_Alimentación infantil,q_Arreglos,q_Arroz,q_Arroz y pasta,q_Atún y otras conservas de pescado,q_Aves y jamón cocido,q_Aves y pollo,q_Azúcar y edulcorante,q_Bacón y salchichas,q_Bases de maquillaje y corrector,q_Berberechos y mejillones,"q_Biberón, chupete y menaje",q_Bollería de horno,q_Bollería envasada,q_Bífidus,q_Cacao soluble y chocolate a la taza,q_Café cápsula y monodosis,q_Café molido y en grano,q_Café soluble y otras bebidas,q_Carne,q_Carne congelada,q_Cerdo,q_Cereales,q_Cerveza,q_Cerveza sin alcohol,q_Champú,q_Chicles y caramelos,q_Chocolate,q_Chopped y mortadela,q_Coloración cabello,q_Colorete y polvos,q_Conejo y cordero,q_Conservas de verdura y frutas,q_Cuidado corporal,q_Cuidado e higiene facial,...,percp_Salazones y ahumados,percp_Sidra y cava,percp_Sopa y caldo,percp_Sushi,percp_Tartas y churros,percp_Tartas y pasteles,percp_Tinto de verano y sangría,percp_Toallitas y pañales,percp_Tomate,percp_Tomate y otros sabores,percp_Tortitas,percp_Turrones,percp_Té e infusiones,percp_Tónica y bitter,percp_Utensilios de limpieza y calzado,percp_Vacuno,percp_Velas y decoración,percp_Verdura,percp_Vino blanco,percp_Vino lambrusco y espumoso,percp_Vino rosado,percp_Vino tinto,percp_Yogures desnatados,percp_Yogures griegos,percp_Yogures líquidos,percp_Yogures naturales y sabores,percp_Yogures y postres infantiles,count_creation_card,count_card_update,count_card_country,datetime_diff_sec,datetime_diff_min,regular_customer,year,month,above_average,quarter,season,datetime_dif_card_sec,datetime_dif_card_min
190654,0,0,5,30,15,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,30,10,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,307.69,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,982.42,0.0,0.00,0.0,0.0,0.00,5,5,4,-554347.0,-9240.0,1,2019,8,1,q3,Summer,5452628.0,90877.0
417404,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,3,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,53.23,0.0,26.45,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,95.16,0.00,0.0,51.61,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,35.16,1,1,1,2705080.0,45084.0,1,2020,1,1,q1,Winter,1311203.0,21853.0
390923,2,1,0,0,2,0,0,17,0,6,5,3,0,0,1,0,0,0,6,0,0,0,0,0,0,5,0,10,1,0,0,0,0,0,0,2,0,0,0,0,...,0.00,0.0,60.62,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,194.69,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,1,1,1,-480686.0,-8012.0,1,2020,1,1,q1,Winter,0.0,0.0
128218,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,45.0,0.0,0.00,0.00,0.0,83.89,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,1,1,1,-131.0,-3.0,1,2019,6,0,q2,Summer,11596445.0,193274.0
450313,6,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.0,0.00,0.0,122.62,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,1,1,1,-213.0,-4.0,1,2020,3,1,q1,Spring,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142345,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,1,1,1,11915424.0,198590.0,0,2019,7,0,q3,Summer,22210894.0,370181.0
477142,1,0,0,0,0,0,0,0,0,0,4,3,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,7,0,0,0,2,0,1,0,1,0,0,0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,83.85,0.0,0.00,0.0,0.0,0.0,0.0,43.48,0.00,0.0,55.28,0.0,0.0,0.0,142.86,0.0,80.12,0.0,67.7,0.00,1,1,1,-79.0,-2.0,1,2020,3,1,q1,Spring,0.0,0.0
405409,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,

# Pre-procesado

### One-Hot Encoding

Utilizaremos la técnica de one-hot encoding.

<img src="figures/oh.png" width="50%">


Seleccionemos las variables categóricas en primer lugar.

In [ ]:
#Mostramos todas las columnas para identificar las categoricas
cols = X2.columns.tolist()
cols

['q_Aceite, vinagre y sal',
 'q_Aceitunas y encurtidos',
 'q_Acondicionador y mascarilla',
 'q_Afeitado y cuidado para hombre',
 'q_Agua',
 'q_Alimentación infantil',
 'q_Arreglos',
 'q_Arroz',
 'q_Arroz y pasta',
 'q_Atún y otras conservas de pescado',
 'q_Aves y jamón cocido',
 'q_Aves y pollo',
 'q_Azúcar y edulcorante',
 'q_Bacón y salchichas',
 'q_Bases de maquillaje y corrector',
 'q_Berberechos y mejillones',
 'q_Biberón, chupete y menaje',
 'q_Bollería de horno',
 'q_Bollería envasada',
 'q_Bífidus',
 'q_Cacao soluble y chocolate a la taza',
 'q_Café cápsula y monodosis',
 'q_Café molido y en grano',
 'q_Café soluble y otras bebidas',
 'q_Carne',
 'q_Carne congelada',
 'q_Cerdo',
 'q_Cereales',
 'q_Cerveza',
 'q_Cerveza sin alcohol',
 'q_Champú',
 'q_Chicles y caramelos',
 'q_Chocolate',
 'q_Chopped y mortadela',
 'q_Coloración cabello',
 'q_Colorete y polvos',
 'q_Conejo y cordero',
 'q_Conservas de verdura y frutas',
 'q_Cuidado corporal',
 'q_Cuidado e higiene facial',
 'q_D

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
#Convert categorical vars that are int to object
X2[['year','month']]= X2[['year','month']].astype(str)
X2.select_dtypes('object')

,client_type,Alpha-3 code,year,month,quarter,season
190654,AND,SGP,2019,8,q3,Summer
417404,AND,ESP,2020,1,q1,Winter
390923,WEB,DEU,2020,1,q1,Winter
128218,WEB,ESP,2019,6,q2,Summer
450313,WEB,SWE,2020,3,q1,Spring
...,...,...,...,...,...,...
142345,AND,ESP,2019,7,q3,Summer
477142,AND,ESP,2020,3,q1,Spring
405409,WEB,ESP,2020,1,q1,Winter
101168,IOS,ESP,2019,5,q2,Spring


In [ ]:
categorical_vars = set(['Alpha-3 code','client_type', 'year','month', 'quarter', 'season' ])
numerical_vars = set(X2.columns) - categorical_vars
categorical_vars = list(categorical_vars)
numerical_vars = list(numerical_vars)

In [ ]:
print(categorical_vars)
print(numerical_vars)

['Alpha-3 code', 'season', 'year', 'month', 'quarter', 'client_type']
['percq_Fitoterapia y parafarmacia', 'total_quantity', 'i_Cereales y galletas', 'p_Mascotas', 'percp_Pizzas y platos preparados', 'i_Agua y refrescos', 'perc_Agua y refrescos', 'perc_Marisco y pescado', 'p_Arroz, legumbres y pasta', 'perc_Cuidado facial y corporal', 'i_Aceite, especias y salsas', 'i_Cuidado facial y corporal', 'percp_Azúcar, caramelos y chocolate', 'percp_Congelados', 'i_Charcutería y quesos', 'p_Fruta y verdura', 'percq_Cereales y galletas', 'percq_Charcutería y quesos', 'i_Fruta y verdura', 'perc_Fitoterapia y parafarmacia', 'percq_Panadería y pastelería', 'percp_Huevos, leche y mantequilla', 'perc_Postres y yogures', 'datetime_dif_card_sec', 'p_Agua y refrescos', 'percp_Zumos', 'perc_Fruta y verdura', 'q_Agua y refrescos', 'percq_Conservas, caldos y cremas', 'p_Maquillaje', 'i_Conservas, caldos y cremas', 'regular_customer', 'i_Carne', 'perc_Congelados', 'p_Zumos', 'p_Azúcar, caramelos y chocolate

Realizamos one hot encoding de las variables categoricas

In [ ]:
ohe = OneHotEncoder(sparse = False)
ohe_fit = ohe.fit(X2[categorical_vars])
X_ohe = pd.DataFrame(ohe.fit_transform(X2[categorical_vars]))
X_ohe.columns = pd.DataFrame(ohe_fit.get_feature_names())

Visualizamos los datos iniciales y los que están con one hot encoding para ver la diferencia

In [ ]:
X[categorical_vars].head()

,month,client_type,season,Alpha-3 code,year,quarter
564,8,AND,Summer,ESP,2018,q3
1056,8,AND,Summer,ESP,2018,q3
1058,8,AND,Summer,ESP,2018,q3
1581,8,WEB,Summer,DNK,2018,q3
1586,8,AND,Summer,ESP,2018,q3


In [ ]:
X_ohe

,"(x0_AGO,)","(x0_AND,)","(x0_ARE,)","(x0_ARG,)","(x0_AUS,)","(x0_AUT,)","(x0_BEL,)","(x0_BGR,)","(x0_BLR,)","(x0_BOL,)","(x0_BRA,)","(x0_CAN,)","(x0_CHE,)","(x0_CHL,)","(x0_CHN,)","(x0_CMR,)","(x0_COL,)","(x0_CRI,)","(x0_CUW,)","(x0_CYP,)","(x0_CZE,)","(x0_DEU,)","(x0_DNK,)","(x0_DOM,)","(x0_ECU,)","(x0_EGY,)","(x0_ESP,)","(x0_EST,)","(x0_FIN,)","(x0_FRA,)","(x0_GBR,)","(x0_GEO,)","(x0_GHA,)","(x0_GNQ,)","(x0_GRC,)","(x0_GTM,)","(x0_HKG,)","(x0_HND,)","(x0_HRV,)","(x0_HUN,)","(x0_IDN,)","(x0_IND,)","(x0_IRL,)","(x0_ISL,)","(x0_ISR,)","(x0_ITA,)","(x0_JAM,)","(x0_JOR,)","(x0_JPN,)","(x0_KAZ,)","(x0_KEN,)","(x0_KHM,)","(x0_KOR,)","(x0_KWT,)","(x0_LBN,)","(x0_LTU,)","(x0_LUX,)","(x0_LVA,)","(x0_MAR,)","(x0_MDA,)","(x0_MEX,)","(x0_MKD,)","(x0_MLT,)","(x0_MNE,)","(x0_MNG,)","(x0_MOZ,)","(x0_MUS,)","(x0_MYS,)","(x0_NLD,)","(x0_NOR,)","(x0_NZL,)","(x0_PAN,)","(x0_PER,)","(x0_PHL,)","(x0_POL,)","(x0_PRI,)","(x0_PRT,)","(x0_PRY,)","(x0_QAT,)","(x0_ROU,)","(x0_RUS,)","(x0_SAU,)","(x0_SGP,)","(x0_SLV,)","(x0_SRB,)","(x0_SVK,)","(x0_SVN,)","(x0_SWE,)","(x0_THA,)","(x0_TTO,)","(x0_TUR,)","(x0_TWN,)","(x0_UKR,)","(x0_URY,)","(x0_USA,)","(x0_VNM,)","(x0_ZAF,)","(x1_Autumn,)","(x1_Spring,)","(x1_Summer,)","(x1_Winter,)","(x2_2018,)","(x2_2019,)","(x2_2020,)","(x3_1,)","(x3_10,)","(x3_11,)","(x3_12,)","(x3_2,)","(x3_3,)","(x3_4,)","(x3_5,)","(x3_6,)","(x3_7,)","(x3_8,)","(x3_9,)","(x4_q1,)","(x4_q2,)","(x4_q3,)","(x4_q4,)","(x5_AND,)","(x5_IOS,)","(x5_WEB,)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

Volvemos a pegar las variables numéricas.

In [ ]:
X2 = pd.concat((X_ohe, X2[numerical_vars].reset_index()), axis=1)
X2

,"(x0_2018,)","(x0_2019,)","(x0_2020,)","(x1_Autumn,)","(x1_Spring,)","(x1_Summer,)","(x1_Winter,)","(x2_q1,)","(x2_q2,)","(x2_q3,)","(x2_q4,)","(x3_AND,)","(x3_IOS,)","(x3_WEB,)","(x4_AGO,)","(x4_AND,)","(x4_ARE,)","(x4_ARG,)","(x4_AUS,)","(x4_AUT,)","(x4_BEL,)","(x4_BGR,)","(x4_BLR,)","(x4_BOL,)","(x4_BRA,)","(x4_CAN,)","(x4_CHE,)","(x4_CHL,)","(x4_CHN,)","(x4_CMR,)","(x4_COL,)","(x4_CRI,)","(x4_CUW,)","(x4_CYP,)","(x4_CZE,)","(x4_DEU,)","(x4_DNK,)","(x4_DOM,)","(x4_ECU,)","(x4_EGY,)",...,p_Pan de horno,i_Neceseres,perc_Isotónico y energético,percq_Golosinas,percq_Lejía y líquidos fuertes,i_Isotónico y energético,"percq_Picos, rosquilletas y picatostes",p_Vino blanco,q_Azúcar y edulcorante,q_Bases de maquillaje y corrector,q_Pescado fresco,p_Limpieza cocina,perc_Harina y preparado repostería,percq_Pan de horno,percp_Helados,q_Limpieza muebles y multiusos,percq_Empanados y elaborados,p_Carne congelada,i_Limpieza muebles y multiusos,purchase_deliv_sec,percp_Tomate,percq_Flan y natillas,p_Turrones,i_Fitoterapia,q_Gazpacho y cremas,p_Cerveza,percq_Pescado fresco,q_Aceitunas y encurtidos,perc_Frutos secos y fruta desecada,"percq_Queso curado, semicurado y tierno",q_Pan de horno,i_Higiene bucal,percq_Pinceles y brochas,percq_Sidra y cava,i_Verdura,percp_Acondicionador y mascarilla,percp_Cerveza sin alcohol,i_Menaje y conservación de alimentos,percq_Aceitunas y encurtidos,percp_Arreglos
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,2.04,0.0,0.0,0.0,0,0,0.0,0.00,0.0,0.00,0.0,0,0.00,0.00,0.00,195447.0,0.0,0.0,0.0,0.0,0,0.00,0.0,0,0.00,0.00,0,52.50,0.0,0.0,0.00,434.07,0.0,0.0,0.00,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,1.47,0.0,0.0,0.0,0,0,0.0,3.10,0.0,0.00,0.0,1,7.35,3.08,1.95,84619.0,0.0,0.0,0.0,0.0,0,0.00,0.0,0,0.00,0.00,0,0.00,0.0,0.0,1.60,0.00,0.0,0.0,0.00,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0,1,0.0,0.00,0.0,0.00,0.0,0,0.00,4.17,0.00,130406.0,0.0,0.0,0.0,0.0,0,7.92,0.0,1,2.02,0.00,0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.86,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0,0,0.0,0.00,0.0,0.00,0.0,0,0.00,0.00,0.00,82719.0,0.0,0.0,0.0,0.0,0,2.22,0.0,1,1.47,0.00,0,0.00,0.0,0.0,4.52,0.00,0.0,0.0,2.78,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0,0,0.0,1.56,0.0,0.00,0.0,0,0.00,0.00,0.00,308536.0,0.0,0.0,0.0,0.0,0,0.00,0.0,0,0.00,0.00,0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157309,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0,0,0.0,0.00,0.0,0.00,0.0,0,0.00,0.00,0.00,110055.0,0.0,0.0,0.0,0.0,0,3.30,0.0,0,0.00,0.00,0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0
157310,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1,0,0.0,0.00,3.7,2.22,0.0,0,0.00,0.00,0.00,758435.0,0.0,0.

In [ ]:
X2= X2.drop(columns= 'index')
X2

,"(x0_2018,)","(x0_2019,)","(x0_2020,)","(x1_Autumn,)","(x1_Spring,)","(x1_Summer,)","(x1_Winter,)","(x2_q1,)","(x2_q2,)","(x2_q3,)","(x2_q4,)","(x3_AND,)","(x3_IOS,)","(x3_WEB,)","(x4_AGO,)","(x4_AND,)","(x4_ARE,)","(x4_ARG,)","(x4_AUS,)","(x4_AUT,)","(x4_BEL,)","(x4_BGR,)","(x4_BLR,)","(x4_BOL,)","(x4_BRA,)","(x4_CAN,)","(x4_CHE,)","(x4_CHL,)","(x4_CHN,)","(x4_CMR,)","(x4_COL,)","(x4_CRI,)","(x4_CUW,)","(x4_CYP,)","(x4_CZE,)","(x4_DEU,)","(x4_DNK,)","(x4_DOM,)","(x4_ECU,)","(x4_EGY,)",...,p_Pan de horno,i_Neceseres,perc_Isotónico y energético,percq_Golosinas,percq_Lejía y líquidos fuertes,i_Isotónico y energético,"percq_Picos, rosquilletas y picatostes",p_Vino blanco,q_Azúcar y edulcorante,q_Bases de maquillaje y corrector,q_Pescado fresco,p_Limpieza cocina,perc_Harina y preparado repostería,percq_Pan de horno,percp_Helados,q_Limpieza muebles y multiusos,percq_Empanados y elaborados,p_Carne congelada,i_Limpieza muebles y multiusos,purchase_deliv_sec,percp_Tomate,percq_Flan y natillas,p_Turrones,i_Fitoterapia,q_Gazpacho y cremas,p_Cerveza,percq_Pescado fresco,q_Aceitunas y encurtidos,perc_Frutos secos y fruta desecada,"percq_Queso curado, semicurado y tierno",q_Pan de horno,i_Higiene bucal,percq_Pinceles y brochas,percq_Sidra y cava,i_Verdura,percp_Acondicionador y mascarilla,percp_Cerveza sin alcohol,i_Menaje y conservación de alimentos,percq_Aceitunas y encurtidos,percp_Arreglos
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,2.04,0.0,0.0,0.0,0,0,0.0,0.00,0.0,0.00,0.0,0,0.00,0.00,0.00,195447.0,0.0,0.0,0.0,0.0,0,0.00,0.0,0,0.00,0.00,0,52.50,0.0,0.0,0.00,434.07,0.0,0.0,0.00,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,1.47,0.0,0.0,0.0,0,0,0.0,3.10,0.0,0.00,0.0,1,7.35,3.08,1.95,84619.0,0.0,0.0,0.0,0.0,0,0.00,0.0,0,0.00,0.00,0,0.00,0.0,0.0,1.60,0.00,0.0,0.0,0.00,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0,1,0.0,0.00,0.0,0.00,0.0,0,0.00,4.17,0.00,130406.0,0.0,0.0,0.0,0.0,0,7.92,0.0,1,2.02,0.00,0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.86,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0,0,0.0,0.00,0.0,0.00,0.0,0,0.00,0.00,0.00,82719.0,0.0,0.0,0.0,0.0,0,2.22,0.0,1,1.47,0.00,0,0.00,0.0,0.0,4.52,0.00,0.0,0.0,2.78,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0,0,0.0,1.56,0.0,0.00,0.0,0,0.00,0.00,0.00,308536.0,0.0,0.0,0.0,0.0,0,0.00,0.0,0,0.00,0.00,0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157309,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0,0,0.0,0.00,0.0,0.00,0.0,0,0.00,0.00,0.00,110055.0,0.0,0.0,0.0,0.0,0,3.30,0.0,0,0.00,0.00,0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0
157310,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,1,0,0.0,0.00,3.7,2.22,0.0,0,0.00,0.00,0.00,758435.0,0.0,0.

### Normalizar

Ahora vamos a normalizar los datos, es decir, llevarlos a media 0 y desviación estándar 1.

<img src="figures/tipify.png" width="50%">

In [ ]:
y = X2['Class']
del X2['Class']

In [ ]:
X_scale = pd.DataFrame(scale(X2))
X_scale.columns = X2.columns
X2 = X_scale
X2.columns = X_scale.columns
X2

,"(x0_2018,)","(x0_2019,)","(x0_2020,)","(x1_Autumn,)","(x1_Spring,)","(x1_Summer,)","(x1_Winter,)","(x2_q1,)","(x2_q2,)","(x2_q3,)","(x2_q4,)","(x3_AND,)","(x3_IOS,)","(x3_WEB,)","(x4_AGO,)","(x4_AND,)","(x4_ARE,)","(x4_ARG,)","(x4_AUS,)","(x4_AUT,)","(x4_BEL,)","(x4_BGR,)","(x4_BLR,)","(x4_BOL,)","(x4_BRA,)","(x4_CAN,)","(x4_CHE,)","(x4_CHL,)","(x4_CHN,)","(x4_CMR,)","(x4_COL,)","(x4_CRI,)","(x4_CUW,)","(x4_CYP,)","(x4_CZE,)","(x4_DEU,)","(x4_DNK,)","(x4_DOM,)","(x4_ECU,)","(x4_EGY,)",...,p_Pan de horno,i_Neceseres,perc_Isotónico y energético,percq_Golosinas,percq_Lejía y líquidos fuertes,i_Isotónico y energético,"percq_Picos, rosquilletas y picatostes",p_Vino blanco,q_Azúcar y edulcorante,q_Bases de maquillaje y corrector,q_Pescado fresco,p_Limpieza cocina,perc_Harina y preparado repostería,percq_Pan de horno,percp_Helados,q_Limpieza muebles y multiusos,percq_Empanados y elaborados,p_Carne congelada,i_Limpieza muebles y multiusos,purchase_deliv_sec,percp_Tomate,percq_Flan y natillas,p_Turrones,i_Fitoterapia,q_Gazpacho y cremas,p_Cerveza,percq_Pescado fresco,q_Aceitunas y encurtidos,perc_Frutos secos y fruta desecada,"percq_Queso curado, semicurado y tierno",q_Pan de horno,i_Higiene bucal,percq_Pinceles y brochas,percq_Sidra y cava,i_Verdura,percp_Acondicionador y mascarilla,percp_Cerveza sin alcohol,i_Menaje y conservación de alimentos,percq_Aceitunas y encurtidos,percp_Arreglos
0,-0.254267,0.748603,-0.652185,-0.604126,-0.567626,2.167851,-0.675740,-0.668465,-0.474116,2.028933,-0.673705,1.303209,-0.626482,-0.729847,-0.002521,-0.016146,-0.010396,-0.023792,-0.015338,-0.011276,-0.007131,-0.004367,-0.003566,-0.006176,-0.027859,-0.013811,-0.020331,-0.063309,-0.029089,-0.002521,-0.01937,-0.008362,-0.010086,-0.007131,-0.005043,-0.030164,-0.006176,-0.009434,-0.014485,-0.006176,...,-0.289436,-0.01963,-0.087689,-0.045466,0.288384,-0.048262,-0.12534,-0.143343,-0.080141,-0.033850,-0.116957,-0.132418,-0.083263,-0.240603,-0.104843,-0.032205,-0.076942,-0.137105,-0.032899,-0.155439,-0.171683,-0.085254,-0.062041,-0.046888,-0.075598,-0.319141,-0.152142,-0.119943,-0.205298,-0.143715,-0.117292,11.665644,-0.020072,-0.049485,-0.238924,10.418175,-0.036837,-0.071893,-0.139980,-0.11488
1,-0.254267,-1.335821,1.533306,-0.604126,-0.567626,-0.461286,1.479859,1.495966,-0.474116,-0.492870,-0.673705,1.303209,-0.626482,-0.729847,-0.002521,-0.016146,-0.010396,-0.023792,-0.015338,-0.011276,-0.007131,-0.004367,-0.003566,-0.006176,-0.027859,-0.013811,-0.020331,-0.063309,-0.029089,-0.002521,-0.01937,-0.008362,-0.010086,-0.007131,-0.005043,-0.030164,-0.006176,-0.009434,-0.014485,-0.006176,...,-0.289436,-0.01963,-0.087689,-0.045466,0.161163,-0.048262,-0.12534,-0.143343,-0.080141,-0.033850,-0.116957,8.982526,-0.083263,-0.240603,-0.104843,0.624231,4.463650,5.053537,0.625993,-0.611975,-0.171683,-0.085254,-0.062041,-0.046888,-0.075598,-0.319141,-0.152142,-0.119943,-0.205298,-0.143715,-0.117292,-0.075161,-0.020072,-0.049485,-0.143612,-0.087425,-0.036837,-0.071893,-0.139980,-0.11488
2,-0.254267,-1.335821,1.533306,-0.604126,-0.567626,-0.461286,1.479859,1.495966,-0.474116,-0.492870,-0.673705,-0.767337,-0.626482,1.370149,-0.002521,-0.016146,-0.010396,-0.023792,-0.015338,-0.011276,-0.007131,-0.004367,-0.003566,-0.006176,-0.027859,-0.013811,-0.020331,-0.063309,-0.029089,-0.002521,-0.01937,-0.008362,-0.010086,-0.007131,-0.005043,33.152645,-0.006176,-0.009434,-0.014485,-0.006176,...,-0.289436,-0.01963,-0.087689,-0.045466,-0.166934,-0.048262,-0.12534,-0.143343,-0.080141,3.107792,-0.116957,-0.132418,-0.083263,-0.240603,-0.104843,-0.032205,-0.076942,6.890485,-0.032899,-0.423364,-0.171683,-0.085254,-0.062041,-0.046888,-0.075598,4.907702,-0.152142,0.382406,0.269609,-0.143715,-0.117292,-0.075161,-0.020072,-0.049485,-0.238924,-0.087425,-0.036837,-0.071893,0.138482,-0.11488
3,-0.254267,0.748603,-0.652185,-0.604126,-0.567626,2.167851,-0.675740,-0.668465,2.109187,-0.492870,-0.673705,-0.767337,-0.626482,1.370149,-0.002521,-0.016146,-0.010396,-0.023792,-0.015338,-0.011276,-0.007131,-0.004

### Split en Train/Validación/Test

Utilizaremos a modo de ejemplo los ratios habitualmente recomendados:

• Train: 70%.

• Validación: 15%.

• Test: 15%.


In [ ]:
perc_values = [0.7, 0.15, 0.15];

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train_rand, X_valtest_rand, y_train_rand, y_valtest_rand = train_test_split(X2, y, test_size=perc_values[1] + perc_values[2], random_state=1);

X_val_rand, X_test_rand, y_val_rand, y_test_rand = train_test_split(X_valtest_rand, y_valtest_rand, test_size= perc_values[2] / (perc_values[1] + perc_values[2]), random_state=1)

Visualizamos el tamaño de los 3 subdatasets

In [ ]:
print('Train data size = ' + str(X_train_rand.shape))
print('Train target size = ' + str(y_train_rand.shape))
print('Validation data size = ' + str(X_val_rand.shape))
print('Validation target size = ' + str(y_val_rand.shape))
print('Test data size = ' + str(X_test_rand.shape))
print('Test target size = ' + str(y_test_rand.shape))

Train data size = (110119, 1036)
Train target size = (110119,)
Validation data size = (23597, 1036)
Validation target size = (23597,)
Test data size = (23598, 1036)
Test target size = (23598,)


#Random Forest


Vamos a aplicar ahora un modelo de Random Forest para intentar predecir la probabilidad de exitus en función de todas las otras variables de las que disponemos en el dataset. Recordemos que la definición de Gradient Boosting es la siguiente:

![imagen.png](attachment:imagen.png)


1) Importar modelo.



In [ ]:
from sklearn.ensemble import RandomForestClassifier 

2) Importar métrica

In [ ]:
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import confusion_matrix

3) Definir el método.

Vamos a entrenar un modelo con algunos parámetros eligidos al azar.

In [ ]:
model = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=None,
                               max_features='auto', max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, 
                               min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0,
                               n_estimators=100, n_jobs=None, oob_score=False, random_state=1, verbose=0, warm_start=False)

4) Llamar al método fit para entrenar el modelo

In [ ]:
#start = timeit.default_timer()

model.fit(X_train_rand, np.array(y_train_rand))

#stop = timeit.default_timer()
#print('Time: ', stop - start)  

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

5) Llamar al método predict para generar las predicciones.

In [ ]:
print('Validation data size = ' + str(X_val_rand.shape))
print('Validation target size = ' + str(y_val_rand.shape))
print('Test data size = ' + str(X_test_rand.shape))
print('Test target size = ' + str(y_test_rand.shape))

# Combinar train y validación
X_test_rand = pd.concat((X_val_rand,X_test_rand), axis = 0)
y_test_rand = np.concatenate((y_val_rand, y_test_rand), axis = 0)

del X_val_rand, y_val_rand

print('Test data size = ' + str(X_test_rand.shape))
print('Test target size = ' + str(y_test_rand.shape))

Validation data size = (23597, 1033)
Validation target size = (23597,)
Test data size = (23598, 1033)
Test target size = (23598,)
Test data size = (47195, 1033)
Test target size = (47195,)


In [ ]:
pred_train_p = model.predict(X_train_rand)
pred_test_p = model.predict(X_test_rand)

6) Calcular métrica usando las predicciones obtenidas en el paso anterior.

In [ ]:
# Calcular métricas de evaluación
auc_train = auc(y_train_rand, pred_train_p);
auc_test = auc(y_test_rand, pred_test_p);
results = pd.DataFrame()
results = results.append(pd.DataFrame(data={'model':['RandomForest (Default)'],'auc_train':[auc_train],'auc_test':[auc_test]}, columns=['model','auc_train', 'auc_test']), ignore_index=True)

In [ ]:
results

,model,auc_train,auc_test
0,RandomForest (Default),1.0,0.892723


In [ ]:
from sklearn.metrics import confusion_matrix 

In [ ]:
cm= confusion_matrix(y_test_rand, pred_test_p)
cm

array([[46905,     1],
       [   62,   227]])

In [ ]:
importances = list(model.feature_importances_)
importances

[0.0005488429825579992,
 0.0003627407241548106,
 0.00034616384651270496,
 0.00018621266076068266,
 0.00084392794226426,
 0.0003741893639403119,
 0.0002857049983938563,
 9.768406533367026e-05,
 8.450322326539844e-05,
 0.00041547119179324485,
 0.00010485203910788803,
 0.00029007186794998265,
 0.00041209564756732086,
 0.000596856751543464,
 0.0004796164161057026,
 0.00044546497536823314,
 0.0010537396385341888,
 0.0002993206987020609,
 0.001208380529468993,
 0.0,
 8.558533355688125e-05,
 0.0,
 0.0,
 0.0,
 9.131308515270861e-05,
 0.0,
 0.0,
 2.7772575120837873e-05,
 0.004525364244113695,
 4.500510081622539e-06,
 5.098974251242631e-05,
 3.073408638107902e-05,
 6.260295120621089e-05,
 0.0,
 0.0,
 0.00016804143535906324,
 0.00016179162138061352,
 1.105213408296495e-05,
 3.451021607666334e-05,
 0.00048625759349052114,
 4.352158157544324e-05,
 0.0,
 6.782339875373414e-07,
 0.0,
 0.013211584520685438,
 1.2012665066045243e-05,
 0.0,
 0.001447733220414113,
 0.0002743883252409997,
 0.0,
 0.0,
 0.0,

In [ ]:
# tuplas de importancia y variable
feature_importances = [(feature, round(importance, 5)) for feature, importance in zip(X, importances)]
# Ordenamos las variables por importancia
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

In [ ]:
feature_importances[0:70]

[('percq_Yogures naturales y sabores', 0.1405),
 ('perc_Tomate y otros sabores', 0.08175),
 ('perc_Pinceles y brochas', 0.02107),
 ('percp_Desodorante', 0.01922),
 ('p_Café soluble y otras bebidas', 0.01849),
 ('percp_Bollería envasada', 0.01585),
 ('perc_Golosinas', 0.01411),
 ('q_Embutido curado', 0.01321),
 ('percq_Fijación cabello', 0.01282),
 ('i_Perfume y colonia', 0.00944),
 ('percq_Tomate y otros sabores', 0.00909),
 ('i_Salazones y ahumados', 0.00894),
 ('percq_Pan de horno', 0.00866),
 ('i_Licores', 0.0083),
 ('percp_Embutido', 0.00734),
 ('percq_Refresco de cola', 0.00592),
 ('i_Platos preparados fríos', 0.00566),
 ('percq_Vino blanco', 0.00483),
 ('percp_Chicles y caramelos', 0.00466),
 ('percq_Detergente y suavizante ropa', 0.00462),
 ('p_Pescado congelado', 0.00459),
 ('percq_Hielo', 0.00455),
 ('q_Cerveza', 0.00453),
 ('p_Sopa y caldo', 0.00449),
 ('p_Depilación', 0.00422),
 ('i_Turrones', 0.00421),
 ('percp_Aves y pollo', 0.0042),
 ('perc_Chicles y caramelos', 0.00402),

##Grid Search

Calculamos el numero de iteraciones totales como la multiplicación de la longitud de los posibles valores de todos los hiperaparámetros.

In [ ]:
bootstrap=  [True, False]
max_depth=  [50,60, None]
max_features= [None]
min_samples_leaf= [1]
n_estimators= [50, 60]

params_values = {'bootstrap': bootstrap,
             'max_depth': max_depth,
             'max_features': max_features,
             'min_samples_leaf': min_samples_leaf,
             'n_estimators': n_estimators}

num_iteraciones = len(params_values['bootstrap']) * len(params_values['max_depth']) * len(params_values['max_features']) * len(params_values['min_samples_leaf']) * len(params_values['n_estimators'])

print('El numero de iteraciones es', num_iteraciones)

El numero de iteraciones es 12


In [ ]:
grid_results = pd.DataFrame();
num_iter = 0
for bootstrap in params_values['bootstrap']:
    for n_estimators in params_values['n_estimators']:
        for max_features in params_values['max_features']:
            for min_samples_leaf in params_values['min_samples_leaf']:
                for max_depth in params_values['max_depth']:
                    # print control iteracion modelo
                    print('Numero de iteración = ' + str(num_iter) +
                          ', parametro bootstrap = ' + str(bootstrap) + 
                          ', parametro number of trees = '  + str(n_estimators) +
                          ', parametro maximo de variables = '  + str(max_features) +
                          ', parametro minumun sample = '  + str(min_samples_leaf) +
                          ', parametro profundidad maxima = '  + str(max_depth) + '\n')
                    
                    # definicion del modelo con sus parametros
                    model = RandomForestClassifier(bootstrap = bootstrap, 
                                                  n_estimators = n_estimators,
                                                  max_features = max_features,
                                                  min_samples_leaf = min_samples_leaf,
                                                  max_depth = max_depth, 
                                                  random_state = 1)

                    # entrenamiento del modelo
                    model.fit(X_train_rand, np.array(y_train_rand))

                    # prediccion del conjunto de validacion
                    pred_val_p = model.predict(X_val_rand)
                    pred_train_p= model.predict(X_train_rand)

                    # Calcular métricas de evaluación
                    auc_train = auc(y_train_rand, pred_train_p)
                    auc_val = auc(y_val_rand, pred_val_p)

                    num_iter += 1;

                    print('Fin de iteración = ' + str(num_iter) +
                          ', parametro bootstrap = ' + str(bootstrap) + 
                          ', parametro number of trees = '  + str(n_estimators) +
                          ', parametro maximo de variables = '  + str(max_features) +
                          ', parametro minumun sample = '  + str(min_samples_leaf) +
                          ', parametro profundidad maxima = '  + str(max_depth)+
                          '. AUC train = '  + str(auc_train) + 
                          ' -  AUC val = '  + str(auc_val)  + 
                          '\n')
                    grid_results = grid_results.append(pd.DataFrame(data={
                                                                            'bootstrap':[bootstrap], 
                                                                            'n_estimators':[n_estimators],
                                                                            'max_features':[max_features],
                                                                            'min_samples_leaf':[min_samples_leaf],
                                                                            'max_depth':[max_depth],
                                                                            'auc_train': [auc_train],
                                                                            'auc_val':[auc_val]},
                                                       columns=['bootstrap', 'n_estimators', 'max_features', 'min_samples_leaf',
                                                                            'max_depth','auc_train','auc_val']), 
                                                       ignore_index=True)


Numero de iteración = 0, parametro bootstrap = True, parametro number of trees = 50, parametro maximo de variables = None, parametro minumun sample = 1, parametro profundidad maxima = 50

Fin de iteración = 1, parametro bootstrap = True, parametro number of trees = 50, parametro maximo de variables = None, parametro minumun sample = 1, parametro profundidad maxima = 50. AUC train = 0.994729907773386 -  AUC val = 0.9130221644793785

Numero de iteración = 1, parametro bootstrap = True, parametro number of trees = 50, parametro maximo de variables = None, parametro minumun sample = 1, parametro profundidad maxima = 60

Fin de iteración = 2, parametro bootstrap = True, parametro number of trees = 50, parametro maximo de variables = None, parametro minumun sample = 1, parametro profundidad maxima = 60. AUC train = 0.9986824769433464 -  AUC val = 0.9130434782608696

Numero de iteración = 2, parametro bootstrap = True, parametro number of trees = 50, parametro maximo de variables = None, para

In [ ]:
grid_results

,bootstrap,n_estimators,max_features,min_samples_leaf,max_depth,auc_train,auc_val
0,True,50,None,1,50,0.994730,0.913022
1,True,50,None,1,60,0.998682,0.913043
2,True,50,None,1,None,0.998682,0.913043
3,True,60,None,1,50,0.994730,0.909420
4,True,60,None,1,60,0.999341,0.913043
5,True,60,None,1,None,0.999341,0.913043
6,False,50,None,1,50,0.995389,0.923572
7,False,50,None,1,60,1.000000,0.923593
8,False,50,None,1,None,1.000000,0.923593
9,False,60,None,1,50,0.995389,0.923593


Comparemos el peor y el mejor resultado

In [ ]:
print(grid_results.iloc[grid_results['auc_val'].idxmin()])
print('-------------------------------------------')
print(grid_results.iloc[grid_results['auc_val'].idxmax()])

bootstrap              True
n_estimators             60
max_features           None
min_samples_leaf          1
max_depth                50
auc_train           0.99473
auc_val             0.90942
Name: 3, dtype: object
-------------------------------------------
bootstrap              False
n_estimators              60
max_features            None
min_samples_leaf           1
max_depth                 60
auc_train                  1
auc_val             0.923615
Name: 10, dtype: object


Veamos la mayor diferencia entre auc_train y auc_val.

In [ ]:
diff = grid_results['auc_train'] - grid_results['auc_val']
grid_results.iloc[diff.idxmax()]

bootstrap               True
n_estimators              60
max_features            None
min_samples_leaf           1
max_depth                 60
auc_train           0.999341
auc_val             0.913043
Name: 4, dtype: object

##Reentrenamos el modelo

In [ ]:
model = RandomForestClassifier(bootstrap = True, n_estimators = 60,
                                                  max_features = None,
                                                  min_samples_leaf = 1,
                                                  max_depth = 60, 
                                                  random_state = 1)

4) Llamar al método fit para entrenar el modelo

In [ ]:
start = timeit.default_timer()

model.fit(X_train_rand, np.array(y_train_rand))

stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  2214.020375426


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=60, max_features=None,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [ ]:
print('Validation data size = ' + str(X_val_rand.shape))
print('Validation target size = ' + str(y_val_rand.shape))
print('Test data size = ' + str(X_test_rand.shape))
print('Test target size = ' + str(y_test_rand.shape))

# Combinar train y validación
X_test_rand = pd.concat((X_val_rand,X_test_rand), axis = 0)
y_test_rand = np.concatenate((y_val_rand, y_test_rand), axis = 0)

del X_val_rand, y_val_rand

print('Test data size = ' + str(X_test_rand.shape))
print('Test target size = ' + str(y_test_rand.shape))

Validation data size = (23597, 1034)
Validation target size = (23597,)
Test data size = (23598, 1034)
Test target size = (23598,)
Test data size = (47195, 1034)
Test target size = (47195,)


5) Llamar al método predict para generar las predicciones.

In [ ]:
pred_train_p = model.predict(X_train_rand)
pred_test_p = model.predict(X_test_rand)

6) Calcular métrica usando las predicciones obtenidas en el paso anterior.

In [ ]:
# Calcular métricas de evaluación
auc_train = auc(y_train_rand, pred_train_p);
auc_test = auc(y_test_rand, pred_test_p);
results = pd.DataFrame()
results = results.append(pd.DataFrame(data={'model':['RandomForest (Optimized)'],'auc_train':[auc_train],'auc_test':[auc_test]}, columns=['model',  'auc_train', 'auc_test']), ignore_index=True)

In [ ]:
results

,model,auc_train,auc_test
0,RandomForest (Optimized),0.999341,0.923833


In [ ]:
from sklearn.metrics import confusion_matrix 

In [ ]:
cm= confusion_matrix(y_test_rand,pred_test_p)
cm     #queremos minimizar los falsos negativos: Ratio de falsos negativos= FN/(FP+FN)

array([[46902,     4],
       [   44,   245]])

###Importancia de las variables

In [ ]:
importances = list(model.feature_importances_)
importances

[4.188705253959344e-05,
 0.0005550938449750708,
 0.0003748899637646627,
 0.0005104727565751851,
 0.0001496448350613255,
 0.0005335161936816234,
 0.0005342673505911575,
 0.00032913018736201394,
 9.072555525503303e-05,
 0.0009746303934639059,
 0.0001828226864870751,
 0.0006857808213322849,
 0.00013136372589984835,
 0.0005354219293317358,
 0.0,
 0.0,
 0.0,
 5.2327334297152645e-05,
 0.0,
 0.0,
 0.0004261172237495159,
 0.0,
 0.0,
 0.0,
 0.0035324521302252743,
 0.0,
 1.9487941510783607e-05,
 0.0,
 0.00023469850309082386,
 0.0,
 0.0,
 0.00039410058460802403,
 0.0,
 0.0,
 0.0,
 0.00047036422714025726,
 1.1286605274142847e-05,
 0.0,
 0.0,
 0.0,
 0.001264963147881159,
 0.0,
 0.0,
 0.0011501131513414824,
 0.00023942916348313235,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0025127328878453213,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0011887538257172859,
 0.0,
 0.0034720392321047557,
 0.0,
 0.0,
 0.0,
 0.0003863665584405666,
 0.0,
 0

In [ ]:
# tuplas de importancia y variable
feature_importances = [(feature, round(importance, 5)) for feature, importance in zip(X, importances)]
# Ordenamos las variables por importancia
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

In [ ]:
feature_importances[0:40]

[('percq_Conservas de verdura y frutas', 0.62653),
 ('p_Refresco de naranja y de limón', 0.08475),
 ('percp_Acondicionador y mascarilla', 0.00891),
 ('i_Depilación', 0.0079),
 ('q_Carne', 0.00353),
 ('q_Leche y bebidas vegetales', 0.00347),
 ('q_Ojos', 0.00341),
 ('p_Coloración cabello', 0.00318),
 ('percp_Café soluble y otras bebidas', 0.00296),
 ('q_Helados', 0.00251),
 ('q_Mermelada y miel', 0.0025),
 ('perc_Yogures griegos', 0.00243),
 ('perc_Pasta y fideos', 0.00228),
 ('percq_Cacao soluble y chocolate a la taza', 0.00228),
 ('percq_Cuidado corporal', 0.0022),
 ('percq_Frutos secos y fruta desecada', 0.00209),
 ('percp_Colorete y polvos', 0.00198),
 ('perc_Harina y preparado repostería', 0.00197),
 ('i_Bollería de horno', 0.00194),
 ('perc_Conservas de verdura y frutas', 0.00194),
 ('percq_Utensilios de limpieza y calzado', 0.00192),
 ('p_Pizzas', 0.00185),
 ('percp_Huevos', 0.00181),
 ('q_Té e infusiones', 0.00167),
 ('percp_Yogures griegos', 0.00145),
 ('i_Vino blanco', 0.00144)

Falta gráfico de la importancia de las variables

In [ ]:
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
# Confusion Matrix
# Accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test_rand,pred_test_p)

0.9980802725758674

In [ ]:
# Recall
from sklearn.metrics import recall_score
recall_score(y_test_rand,pred_test_p, average=None)

array([0.99978232, 0.76156584])

In [ ]:
# Precision
from sklearn.metrics import precision_score
precision_score(y_test_rand,pred_test_p, average=None)

array([0.99828671, 0.96179775])